In [55]:
mi_tablero = Tablero.imprimir_tablero
print(mi_tablero)

<function Tablero.imprimir_tablero at 0x000002B662EBFC40>


In [ ]:
barcos = [
    {'nombre': 'portaaviones', 'tamano': 5},
    {'nombre': 'acorazado', 'tamano': 4},
    {'nombre': 'crucero', 'tamano': 3},
    {'nombre': 'submarino', 'tamano': 3},
    {'nombre': 'destructor', 'tamano': 2}
]

# print(barcos[0].keys())

class Tablero:

    def __init__(self, tablero, barcos):
        self.tablero = [['~' for _ in range(10)] for _ in range(10)]
        self.barcos = []

    def colocar_barco(self, barco, posicion, orientacion):
        fila, columna = posicion
        tamano = barco['tamano']

    if orientacion == 'H' and columna + tamano > 10:
        print('Error: El barco no cabe en esta posición')
        return False
    def mostrar_tablero(self):
        

    def todos_barcos_hundidos(self):


dict_keys(['portaaviones'])
